In [1]:
import sys
import tensorflow as tf
import numpy as np
sys.path.append("../")
from tfimps import Tfimps

# Tfimps calculations for transverse field Ising

In [3]:
phys_d = 2
bond_d = 16

# Pauli matrices. For now we avoid complex numbers
X = tf.constant([[0,1],[1,0]], dtype=tf.float64)
iY = tf.constant([[0,1],[-1,0]], dtype=tf.float64)
Z = tf.constant([[1,0],[0,-1]], dtype=tf.float64)

I = tf.eye(phys_d, dtype=tf.float64)

XX = tf.einsum('ij,kl->ikjl', X, X)
YY = - tf.einsum('ij,kl->ikjl', iY, iY)
ZZ = tf.einsum('ij,kl->ikjl', Z, Z)
X1 = (tf.einsum('ij,kl->ikjl', X, I) + tf.einsum('ij,kl->ikjl', I, X)) / 2

In [4]:
# Ising Hamiltonian (at criticality). Exact energy is -4/pi=-1.27324...
h_ising = - ZZ - X1

imps = Tfimps(phys_d, bond_d, symmetrize=True, hamiltonian=h_ising)

train_op = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(imps.variational_e)

with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())

    for i in range(50):
        print(sess.run([imps.variational_e, train_op])[0])

-1.0243753723737807
-1.0351221694985309
-1.04809965905192
-1.0632659302948249
-1.0804973908404347
-1.099556064680165
-1.1200891772621728
-1.1416123122595294
-1.1634928724211033
-1.184964824670064
-1.2051748756762435
-1.2232494975372095
-1.2383762686420967
-1.2499005541464878
-1.2574356981860701
-1.2609722647097255
-1.2609525913335844
-1.258260493211469
-1.2540872012483655
-1.249688027370728
-1.2461139633456084
-1.2440294055618044
-1.2436719967886642
-1.2449255941088435
-1.2474412541716773
-1.2507565842835542
-1.2543918326025891
-1.2579188967716184
-1.2610057774500036
-1.2634395338245625
-1.2651302402359352
-1.2660986707890176
-1.2664516609762415
-1.266350522614445
-1.2659784225655948
-1.2655118510631642
-1.2650994476165904
-1.2648492727338052
-1.264823868435943
-1.2650415072536931
-1.2654818246326327
-1.266094289283879
-1.2668083631316025
-1.2675445117022084
-1.2682253380493327
-1.2687860239727935
-1.2691830531907238
-1.2693999999175576
-1.269449200238813
-1.2693685446564265


## AKLT 

In [2]:
phys_d = 3
bond_d = 2

# Follow Annals of Physics Volume 326, Issue 1, Pages 96-192.
# Note that even though the As are not symmetric, the transfer matrix is.

Aplus = np.array([[0, 1/np.sqrt(2)], [0, 0]])
Aminus = np.array([[0, 0], [-1/np.sqrt(2), 0]])
A0 = np.array([[-1/2, 0], [0, 1/2]])
A_matrices = np.array([Aplus, A0, Aminus])

# Spin 1 operators.

X = tf.constant([[0, 1, 0 ], [1, 0, 1], [0, 1, 0]], dtype=tf.float64) / np.sqrt(2)
iY = tf.constant([[0, -1, 0 ], [1, 0, -1], [0, 1, 0]], dtype=tf.float64) / np.sqrt(2)
Z = tf.constant([[1, 0, 0], [0, 0, 0], [0, 0, -1]], dtype=tf.float64)

XX = tf.einsum('ij,kl->ikjl', X, X)
YY = - tf.einsum('ij,kl->ikjl', iY, iY)
ZZ = tf.einsum('ij,kl->ikjl', Z, Z)

hberg = XX + YY + ZZ
h_aklt = hberg + tf.einsum('abcd,cdef->abef', hberg, hberg) / 3

aklt = Tfimps(phys_d, bond_d, hamiltonian=h_aklt)

In [3]:
train_op = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(aklt.variational_e)

with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())

    for i in range(200):
        print(sess.run([aklt.variational_e, train_op])[0])

1.2204689514921285
1.1190814234288333
1.0415929652292082
0.9841182732563538
0.9410611023350592
0.907986402433856
0.8818434633672216
0.8605937031236712
0.8428610292694706
0.827695310535795
0.8144260001665247
0.8025733858408988
0.7917936923717946
0.7818434875405522
0.7725548819346362
0.7638168140941948
0.7555604095330327
0.7477472141553938
0.7403598915537408
0.7333948562602395
0.7268566007031514
0.720753295847245
0.7150934815993178
0.7098835540935692
0.7051259657608998
0.7008179735661022
0.696950923141836
0.6935100047398132
0.6904744429672385
0.6878181110113871
0.685510497161164
0.6835179393700933
0.6818050389900423
0.6803361199257293
0.6790766046644485
0.6779942033489501
0.6770598232308905
0.6762481533351848
0.675537913656809
0.6749118009427532
0.6743561812834921
0.6738605994169204
0.6734171784532191
0.6730199802682076
0.6726643876540609
0.6723465570295206
0.6720629753740766
0.6718101430071443
0.6715843874022263
0.6713818019075694
0.6711982920969609
0.6710297041650235
0.6708720046122609